In [14]:
# Importing libraries
import pandas as pd
from IPython.display import display, HTML
from sklearn.preprocessing import MinMaxScaler
import os
import re


base_dir = os.getcwd()

In [9]:
class excelFile():
    """
    This class takes excel files which contain census data on certain categories as objects 
    and uses methods on them to convert them into the correct format for clustering

    Class Attributes
    ----------------

    column_idx_counter : int
        keeps track of the index of the columns as the main df is created
    
    previous_column_idx : int
        Keeps track of the index of the columns as the main df is created

    df_index_dict : dict
        A dictionary where the keys are the data categories and the values are the columns they are located in

    df_main : DataFrame
        A dataframe which stores all the data categories to be clustered

    df_main_no_norm : DataFrame
        Same as df_main except there is no normalization applied, used for displaying post clustered stats
    """
    # Stores the index's of each category for the final dataframe
    column_idx_counter = 0
    previous_column_idx = 0
    df_index_dict = {"Lower tier local authorities" : 0}
    catergory_list = []
    # Two dataframes are made, one will be normalised for clustering, the other is non-normalised as it is used for statistics
    LLA_dir = os.path.join(base_dir, "LLA's.xlsx")
    df_main : pd.DataFrame = pd.read_excel(LLA_dir)
    df_main_no_norm : pd.DataFrame = pd.read_excel(LLA_dir)


    def __init__(self, scr):
        """
        Initializes the class object with all relevant parameters

        Parameters
        ----------
            scr : str
                The name of the file
            df : DataFrame
                The dataframe which will stores the contents of the class object

        Returns
        -------
           None
        """
        self.scr: str = scr
        self.df : pd.DataFrame 
    
    def importData(self):
       """
       Imports the excel file into the script as a pandas dataframe
       Drops any NaN values

       Paramters
       ---------
           n/a

        Returns
        -------
           None
       """
       excel_file = pd.ExcelFile(self.scr)
       self.df = pd.read_excel(excel_file, sheet_name="Dataset")
       self.df = self.df.dropna()
        
    def dropColumn(self, index):
        """
        Removes a column from the dataframe

        Parameters
        ----------
            index : int
                The index where the column is located
        
        Returns
        -------
           None
        """
        self.df.drop(self.df.columns[index], axis=1, inplace=True)

    def concatenateDataFrame(self, other_df, axis, reset_index):
        """
        Concatenates two dataframes together

        Parameters
        ----------
            other_df : DataFrame
                The dataframe to be concatenated with

            axis : int
                1 or 0, determines if horizontal or vertical concatenation

            reset_index : bool
                determines if the index is reset after concatenation

        Returns
        -------
           None
        """
        self.df = pd.concat([self.df,other_df], axis=axis, ignore_index=reset_index)     

    def findPivotColumns(self):
        """
        Uses regex expression to parse the dataframe to find the categories columns that will be pivoted

        Parameters
        ----------
            n/a

        Returns
        -------
           None
        """
        self.pivot_columns = []
        # This regex value finds our pivot columns
        # For each data category there are two columns one with the data the other with a data category code e.g 1-8
        # For the pivot's columns we just want the columns with the data
        # The difference between the two columns names is that the category code has the word code in it after a )
        # This regex value differentiates them as it only pick columns that dont contain the word 'code' after a )
        regex = r".*\)(?!.*Code).*"
        for column in self.df.columns:
            if re.search(regex, column):
                self.pivot_columns.append(column)
            
    def convertPivotTable(self, groups, catergories, observations):
        """
        Converts a dataframe into a pivot table

        Parameters
        ----------
            groups : str
                The name of the column which the pivot table will be grouped on

            categories : list 
                A list of columns names (strings) which are to be pivoted

            observations : str
                The name of the column where the observations are recorded for each category

        Returns
        -------
           None
        """
        self.df = pd.pivot_table(self.df, index=groups, columns= catergories, values= observations)
        self.df = self.df.reset_index()

    def dropDoesNotApplyRows(self):
        """
        Drops rows and columns from the dataframe where the category is "Does not apply"

        Parameters
        ----------
            n/a

        Returns
        -------
           None
        """
        for columns in self.pivot_columns:
            self.df = self.df[self.df[columns] != "Does not apply"]

    def addToMainDataFrame(self):
        """
        Adds the dataframe belonging to the class object (self.df) to the class variable df_main

        Parameters
        ----------
            n/a

        Returns
        -------
           None
        """
        # Here we add individual data categories to the main df
        # The length of columns of the data categories is used to determine the index of them in the main df
        # The class variables is updated to keep track of the index of the most recent category
        length = len(self.df.columns) - 1
        df_index_num = length + excelFile.column_idx_counter
        excelFile.df_index_dict[self.pivot_columns[0]] = [excelFile.previous_column_idx+1,df_index_num+1]
        excelFile.catergory_list.append(self.pivot_columns[0])
        excelFile.column_idx_counter += length
        excelFile.previous_column_idx = df_index_num 
        excelFile.df_main = pd.merge(excelFile.df_main, self.df, on="Lower tier local authorities", how="inner")

    def addToMainDataFrameNoNorm(self):
        """
        Adds the dataframe belonging to the class object (self.df) to the class variable df_main_no_norm

        Parameters
        ----------
            n/a

        Returns
        -------
           None
        """
        # This is the same as the previous function except it is used to add non-normalised data
        # This is used to for when we want to show statistics about certain data categories for the user
        local_auth = self.df["Lower tier local authorities"]
        int_attr = self.df.iloc[:,1:]
        self.df = pd.concat([local_auth,int_attr],axis=1)
        excelFile.df_main_no_norm = pd.merge(excelFile.df_main_no_norm, self.df, on="Lower tier local authorities", how="inner")

    def sortDataFrame(self, column_name):
        """
        Sorts the dataframe by a specified column

        Parameters
        ----------
            column_name: str
                The column name on which to sort the dataframe

        Returns
        -------
           None
        """
        self.df = self.df.sort_values(by=column_name).reset_index(drop=True)

    def computePerCapita(self, per):
        """
        Converts all int values in the dataframe to be per capita depending on the respondents from that LLA
        The number of observations from each LLA is imported as a df from an excel file

        Parameters
        ----------
            per : int
                Determines how many people each values should represent e.g 100 means every values is per 100 observations

        Returns
        -------
           None
        """
        # This changes all data values in the dataframe to be per a certain number
        # In this code we use per 1000 households
        # The imported excelfile contains the number of households which responded in each LLA
        respondants_dir = os.path.join(base_dir, "RespondantsCensus.xlsx")
        obs = excelFile(respondants_dir)
        obs.importData()
        obs.dropColumn(0)
        obs.sortDataFrame("Lower Tier Local Authorities")
        local_auth = self.df["Lower tier local authorities"]
        self.df = self.df.iloc[:,1:].div(obs.df["Observation"]/per, axis=0)
        self.df.insert(loc=0, column="Lower tier local authorities", value=local_auth)

    def normalizeValues(self):
        """
        Normalizes the dataframe

        Parameters
        ----------
            n/a

        Returns
        -------
           None
        """
        # This normalises the values as this help with clustering
        # This is because it stops higher magnitude attributes from having more of an effect than lower magnitude attributes
        local_auth = self.df["Lower tier local authorities"]
        int_attr = self.df.iloc[:,1:]
        normalized_attr = pd.DataFrame(MinMaxScaler().fit_transform(int_attr), columns=int_attr.columns)
        self.df = pd.concat([local_auth,normalized_attr],axis=1)





In [10]:
excelFiles_dir = os.path.join(base_dir, "excelFiles")
os.chdir(excelFiles_dir)

def excelLoop():
    """
    Sets up the main dataframe to be used in clustering
    Iterates through excel files using os modules and creates class objs from excelFile class
    Using class methods to convert individual dataframes into specific format

    Parameters
    ----------
        n/a

    Returns
    -------
        None
    """
    # This is the main loop to pre-process the excel data
    # It handles multiple processes, imports, finds the pivot columns, gets rid of NaN values, drops the first redundant column
    # sorts the data frames alphabetically on LLAs,coverts the data to a pivot table, changes the data to per 1000 households,
    # adds the data to a non-normalised dataframe, normalises the data, adds it to the main dataframe
    for filename in os.listdir():
        obj = excelFile(filename)
        obj.importData()
        obj.findPivotColumns()
        obj.dropDoesNotApplyRows()
        obj.dropColumn(0)
        obj.sortDataFrame("Lower tier local authorities")
        obj.convertPivotTable("Lower tier local authorities", obj.pivot_columns, "Observation")
        obj.computePerCapita(1000)
        obj.addToMainDataFrameNoNorm()
        obj.normalizeValues()
        obj.addToMainDataFrame()



   

       

 
        
        

In [11]:
excelLoop()
display(excelFile.df_main)

,Lower tier local authorities,A caravan or other mobile or temporary structure,"Flat, maisonette or apartment",Whole house or bungalow: Detached,Whole house or bungalow: Semi-detached,Whole house or bungalow: Terraced,"No adults, or one adult and one or more children",One-person household: One adult aged 66 years or over,One-person household: One person aged 65 years or under,Three or more adults: No children,...,Mains gas only,No central heating,Oil only,Other central heating only,Renewable energy only,Solid fuel only,Tank or bottled gas only,Two or more types of central heating (including renewable energy),Two or more types of central heating (not including renewable energy),Wood only
0,Adur,0.269943,0.189673,0.334605,0.774599,0.375896,0.385148,0.809311,0.082015,0.398024,...,0.933795,0.061298,0.003135,0.100601,0.044201,0.008011,0.001947,0.073588,0.102581,0.028660
1,Allerdale,0.138366,0.042082,0.523782,0.687429,0.538714,0.463256,0.718533,0.226585,0.283711,...,0.801717,0.053682,0.219521,0.109191,0.191694,0.497201,0.198393,0.089957,0.501336,0.323589
2,Amber Valley,0.053217,0.026583,0.677435,0.743576,0.356757,0.448549,0.617802,0.181798,0.311182,...,0.939222,0.032912,0.042456,0.054072,0.082514,0.223799,0.054666,0.077347,0.316490,0.086336
3,Arun,0.417815,0.179558,0.614881,0.505316,0.347385,0.326471,0.892877,0.071096,0.357866,...,0.896467,0.053341,0.027782,0.085981,0.092171,0.019531,0.087436,0.089645,0.144895,0.039575
4,Ashfield,0.029496,0.041345,0.559512,0.854587,0.349282,0.581008,0.538910,0.186165,0.347120,...,0.973739,0.013144,0.012571,0.169634,0.054309,0.376697,0.025108,0.047456,0.221064,0.041840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,Wrexham,0.057701,0.076773,0.584257,0.763666,0.343431,0.693373,0.601187,0.189966,0.376751,...,0.857539,0.028339,0.195654,0.094173,0.115964,0.153916,0.141552,0.070533,0.283168,0.122419
327,Wychavon,0.476540,0.064419,0.767691,0.623671,0.280603,0.345538,0.638966,0.075494,0.372239,...,0.746789,0.032577,0.254128,0.062845,0.246187,0.123712,0.502789,0.149203,0.387988,0.114966
328,Wyre,0.709159,0.065470,0.552978,0.817062,0.305640,0.435581,0.833368,0.129782,0.300318,...,0.916917,0.044543,0.067576,0.069691,0.079778,0.077746,0.164774,0.053684,0.207162,0.104496
329,Wyre Forest,0.743613,0.089720,0.543883,0.753911,0.329014,0.428473,0.741318,0.160288,0.335454,...,0.869964,0.058497,0.066107,0.073168,0.058306,0.087091,0.299434,0.045314,0.373364,0.092988


In [12]:
# This exports the final dataframes as csv files
# These are then placed into the flask website to be used for clustering
os.chdir(base_dir)
excelFile.df_main.to_csv("main_df.csv")
excelFile.df_main_no_norm.to_csv("main_df_no_norm.csv")